# Parameters generator

The SM will be defined as pandas file in JSON format.
In this file it will be shown the definition of such a file from a dictionary

## Load Model file

In [1]:
pwd

'/home/restrepo/tmp/BSM-Submodules'

In [2]:
MODEL_PATH="./SARAH/Models/SM/SM.m"

In [3]:
f=open(MODEL_PATH,'r')
smf=f.read()
f.close()

In [4]:
smf

'Off[General::spell]\n\nModel`Name = "SM";\nModel`NameLaTeX ="Standard Model";\nModel`Authors = "F.Staub";\nModel`Date = "2014-11-06";\n\n(* 2013-01-24: changed normalization of lambda term to convention of hep-ph/0207271 *)\n(* 2013-06-24: using new name conventions (without inital "S" and "F" for scalar and matter fields) *)\n(* 2013-09-01: changing to new conventions for FermionFields/MatterFields *)\n(* 2014-11-06: Changed sign in Lagrangian to fit standard conventions *)\n(* 2016-05-03: Changed sign of Yu *)\n\n\n\n(*-------------------------------------------*)\n(*   Particle Content*)\n(*-------------------------------------------*)\n\n(* Gauge Groups *)\n\nGauge[[1]]={B,   U[1], hypercharge, g1,False};\nGauge[[2]]={WB, SU[2], left,        g2,True};\nGauge[[3]]={G,  SU[3], color,       g3,False};\n\n\n(* Matter Fields *)\n\nFermionFields[[1]] = {q, 3, {uL, dL},     1/6, 2,  3};  \nFermionFields[[2]] = {l, 3, {vL, eL},    -1/2, 2,  1};\nFermionFields[[3]] = {d, 3, conj[dR],     1

## General definition

In [5]:
#TODO: Initialize objects
particles=[]
particlessons=[]

Fields=['Gauge','FermionFields','ScalarFields']
NAME='DEFINITION'
KEY='EWSB'
# Get also particles from
#NAME[KEY] dictionary!

In [6]:
import pandas as pd
import numpy as np
# import re, cmdlike as cmd, OrdereDict, json
from SARAH import *
pd.set_option('display.max_colwidth',200)

In [7]:
%%writefile SARAH.py
import re
import cmdlike as cmd
from collections import OrderedDict
import json
import numpy as np
import copy


def get_particles(fdotm,Fields,NAME,KEY,particles,particlessons):
    '''
    Extract particles from  SARAH Model files by using the information
    from: 
    * Fields: list of lists
    * NAME[KEY] dictionary 
    '''
    #i=1
    #Field=Fields[i]
    for Field in Fields:
        for f in cmd.grep(Field,fdotm).split('\n'):
            particle={}
            if not re.search('^\s*\(\*',f):
                #Fix components fields
                ff=f.split('{')
                if len(ff) ==3:
                    fff=ff[2].split('}')
                    f=ff[0]+'{'+ff[1]+fff[0].replace(',','::')+''.join(fff[1:])+'};'
                g=re.search('%s\[\[[0-9]+\]\]\s*=\s*\{(.*)\s*\}\s*;\s*' 
                    %Field,f)

                if g:
                    try:
                        fp=g.groups()[0].split(',')
                    except 'KeyError':
                        fp=[]
                    if len(fp)>0:
                        print(g.groups(),Field,len(fp))
                        particle['Field']=fp[0].strip()
                        particle['Parents']=None
                        particle['Properties']={}
                    if Field=='Gauge' and len(fp)>=5:
                        particle['Field']='V'+particle['Field']
                        particle['Properties']['Group']=fp[1]
                        particle['Definition']='GaugeES'
                        particle['Properties']['Index']=fp[2]
                        particle['Properties']['Coupling']=fp[3]
                        particle['Properties']['SSB']=fp[3]
                        particle['Properties']['Lorentz']='Vector'
                        print(particle)
                    else: 
                        if len(fp)>=6:
                            particle['Properties']['NF']=fp[1].strip()
                            particle['Properties']['Groups']=[ 
                                x.strip().replace(';','') for x in fp[3:]]
                            particle['Definition']='WeylFermionAndIndermediate'
                            if Field=='FermionFields':
                                particle['Properties']['Lorentz']='WeylFermion'
                            elif Field=='ScalarFields':
                                particle['Properties']['Lorentz']='Scalar'
                            print("**********")
                            sons=re.sub('conj\[(\w+)\]',r'\1', fp[2] ).split('::')
                            if len(sons)>1:
                                particle['Properties']['multiplet']=[s.strip() for s in sons]

                            print(fp[2],sons)
                            for s in sons:
                                print(s)
                                particleson={}
                                particleson['Properties']={}
                                particleson['Field']=s.strip()
                                if Field=='FermionFields':
                                    particleson['Definition']='WeylFermionAndIndermediate'
                                    particleson['Properties']['Lorentz']='WeylFermion'
                                elif Field=='ScalarFields':
                                    particleson['Definition']='GaugeES'
                                    particleson['Properties']['Lorentz']='Scalar'
                                particleson['Parents']=particle['Field']
                                particleson['Properties']['NF']=particle['Properties']['NF']
                                particlessons=particlessons+[particleson]
                            print("**********")

                    particles.append(particle)
    particles=particles+particlessons
    return particles

def sarahlist_to_python(strl,only_extract=False,DEBUG=False):
    '''
    Convert a string with a SARAH list of rotations into
    a Python object
    '''
    if only_extract:
        return strl
    #General transformations
    nl=re.sub(';\s*$','', #Drop final semicolon
               strl)
    if ( re.search('^\s*\t*\{.*\{.*\{.*->True',strl ) or 
       re.search('^\s*\t*\{.*\{.*\{.*->False',strl ) ):
        nl=re.sub( '([\w]+)([,:])',r'"\1"\2', # Keep True and False
              re.sub( '[\s\t]+','', # strip
              re.sub( '\s*->\s*',':', # to python dict                  
                  nl))).replace('}},{','}],['
                      ).replace('{{','[['
                      ).replace('}}}','}]]'
                      )        
    elif nl.find('->')>-1:
        nl=re.sub( '(\w+\[*\w+\]*)',r'"\1"' ,
           re.sub('\s*:\s*\{([\s\w,\[\]]+)\}',r':[\1 ]', # to python value lists of dict key
           re.sub( '\s*->\s*',':', # to python dict                  
              nl)))
    else:
        nl= re.sub( '\{','[',
            re.sub( '\}',']',                   
            re.sub('([\w\[\]\/\\\]+)',r'"\1"',  #to python_lists
           nl)))
    if DEBUG:    
        print(nl)
        print("*"*50)
    return eval(nl)

def extract_code_block(fdotm,pattern,pattern_start,pattern_end,only_extract=False):
    dsbd={}
    start=False
    startlist=False
    fields=''
    dsbt=''
    for f in fdotm.split('\n'):
        dsb=re.search(pattern,f)
        if dsb:
            #Capture EWSB DEFINITION 
            dsbt=dsb.groups()[0]
            start=True

        if start and f.find(pattern_start)>-1:
            startlist=True

        if startlist:
            fields=fields+f
        if re.search(pattern_end,f):
            start=False
            startlist=False
            if fields:
                srl=sarahlist_to_python(fields.split('=')[-1],only_extract)
                dsbd.update( {dsbt:srl} )
                
                fields=''
    return dsbd

def parse_mathematica_list_of_list(fdotm,NAME='DEFINITION',KEY='EWSB'):
    '''
    Parse mathematica list with the structure:
    
      NAME[KEY][KEYS]={
                             LIST
                             };
    
    and generate a dictionary with KEYS
    '''
    pattern='%s\[%s\]\[(\w+)\]' %(NAME,KEY)
    pattern_start='{'
    pattern_end='\}\s*;'
    only_extract=False
    dsbd=extract_code_block(fdotm,pattern,pattern_start,pattern_end,only_extract)
    return dsbd

def bidiagonal(w,k='MatterSector'):
    weyl={}
    weyl['left_intr'  ]=w[0][0] #list
    weyl['right_intr' ]=w[0][1] #list
    weyl['left_mass' ]=w[1][0][0] 
    weyl['left_rota'  ]=w[1][0][1]
    weyl['right_mass']=w[1][1][0]
    weyl['right_rota' ]=w[1][1][1]
    if k=='MatterSector':
        weyl['lorentz' ]='WeylFermion'
        #If Diagonal then Scalar or Majorana
    return weyl

def diagonal(s,k='GaugeSector'):
    symm={}
    symm['intr']=s[0]
    symm['mass']=s[1]
    symm['rota']=s[2]
    if k=='GaugeSector':
        symm['Lorentz']='Vector'
    return symm    

def get_vev(v,k='VEVs'):        
    Vev={}
    Vev['Complex']=v[0]
    Vev['vev']=v[1][0]
    Vev['Imaginary']=v[2][0]
    Vev['Real']=v[3][0]
    Vev['vev_coeff']=v[1][1]
    Vev['Imaginary_coeff']=v[2][1]
    Vev['Real_coeff']=v[3][1]
    Vev['Lorentz']='Scalar'
    return Vev

def order_dict_by(d,element='Description'):
    l=list(d.keys())
    od=d
    if element in l:
        l.remove(element)
        l=[element]+l
        od=OrderedDict()
        for k in l:
            od[k]=d[k]
    return od

def to_math(SM,file,definitions='ParticleDefinitions',
               not_str    =['DependenceNum','Dependence'],
               list_or_str=['OutputName','Mass','Goldstone']):
    '''
    Write mathematica file from `SM` dictionary for either
     * particle.m:  definitions='ParticleDefinitions'
     * parameter.m: definitions='ParameterDefinitions'
    
    `not_str` is the list of keys to be printed without double quotes quotes
    `list_or_str` is the list of keys which may be either a string or a list of strings
    
    INPUT FORMAT in json (json.dumps(dict))
    {"PROPERTY1": {"PARAMETER1": {"KEY1":"STR_VALUE"},
                                {"KEY2": PYTHON_LIST},
                  "PARAMETER2": ...},
     "PROPERTY2":...           }
    
    If PROPERTY=Properties, it is ignored in the output file.
    '''
    f=open(file,'w')
    for c in SM.columns:
        if definitions=='ParticleDefinitions':
            f.write('{}[{}] = {{\n'.format(definitions,c))
        else:
            f.write('{} = {{\n'.format(definitions))

        cindex=SM[c].dropna().index
        csep=','
        for p in cindex:
            SMcp=order_dict_by(SM[c][p],element='Description')
            if p==list(cindex)[-1]:
                csep=''
            f.write('    {{{}, {{'.format(p))
            cpkeys=SMcp.keys()
            sangria=' '
            sep=','
            nl='\n'
            #TODO: Be sure that Description will be printed first
            for k in cpkeys:
                if k!=list(cpkeys)[0]:
                    sangria='           '
                if k==list(cpkeys)[-1]:
                    sep=''
                    nl=''
                #print list of strings with double quotes
                cp='{}{} -> '.format(sangria,k)
                smcpk=SMcp[k]
                if k in not_str:
                    smcpk='{}'.format(smcpk)
                elif type(smcpk)==str and smcpk.find('[')==-1:
                    #Special cases
                    if k not in list_or_str:
                        smcpk='"{}"'.format(smcpk)
                    else:
                        smcpk='{}'.format(smcpk)                    
                else:
                    if smcpk:
                        smcpk=json.dumps( smcpk ).replace('[','{').replace(']','}')

                f.write('{}{}{}{}'.format(cp,smcpk,sep,nl))

            f.write('}}}}{}\n'.format(csep))
        f.write('};\n\n')
    f.close()
    f=open(file,'r')
    return f.read()
    f.close()
    
#Particle definitions
def rotations_to_particles(rotations,key='EWSB',lr='',sep='',DEBUG=False):
    if not lr and not sep:
        typekey='Diagonal'
    elif lr=='left' and sep=='_':
        typekey='Bidiagonal'
    elif lr=='right' and sep=='_':
        typekey='Bidiagonal'
    else:
        print('WARNING: Not yet implemented')
    newparticles=[]
    for k in rotations[key].keys():
        for bd in rotations['EWSB'][k]:
            if k==typekey:
                interaction='{}{}intr'.format(lr,sep)
                if type(bd.get(interaction))==list:
                    if DEBUG: print(bd)
                    for i in  range(len(bd.get(interaction))):
                        if type( bd.get('{}{}mass'.format(lr,sep) ) )==list:
                            fm=bd.get('{}{}mass'.format(lr,sep))[i]
                        else:
                            fm=bd.get('{}{}mass'.format(lr,sep))
                        f=re.sub('\s*\]\s*','',re.sub('conj\s*\[\s*','',
                                                    fm))
                        if f not in [ d.get('Field') for d in newparticles if type(d)==dict]:
                            #left particle
                            particle={'Properties':{}}
                            particle['Field']   =f
                            particle['Parents'] =bd.get(interaction)[i]
                            particle['rotation']=bd.get('{}{}rota'.format(lr,sep))
                            particle['Properties']['Mass_basis']=bd.get('{}{}mass'.format(lr,sep))
                            particle['Properties']['Interaction_basis']=bd.get(interaction)
                            particle['Block']='MatterField'
                            if lr:
                                particle['Definition']='WeylFermionAndIndermediate'
                                particle['Properties']['Chirality']=lr
                                particle['Properties']['Lorentz']='WeylFermion'
                            if bd.get('Lorentz')=='Vector':
                                particle['Definition']='EWSB'
                                particle['Block']='GaugeSector'
                                particle['Properties']['Lorentz']='Vector'
                            else:
                                if not lr:
                                    print('CODE for Scalar or Fermion here')
                            newparticles.append(particle)
                            #print('*'*50)
                            #print(particle)
                            #print('*'*50)
                            if DEBUG: print(i,bd.get('{}{}mass'.format(lr,sep)),particle)
                else:
                    print('No list in mass basis for diagonal particle, please check:',k,p)
    return newparticles

def vev_to_particles(d,cp="Real"):
    if cp!='Real' and cp!='Imaginary':
        return "Error: undefined CP"
    particle={'Properties':{}}
    particle['Field']     =d.get(cp)
    particle['Parents']   =d.get('Complex')
    particle['Definition']='EWSB'
    particle['Block']='VEVs'
    particle['Properties']['Lorentz']='Scalar'
    particle['Properties']['vev']=d.get('vev')
    particle['Properties']['CP']=cp
    particle['Properties']['Coefficient']=d.get('{}_coeff'.format(cp))
    return particle

def spinor_to_particles(dict_of_spinors,rotations,f):
    particle={'Properties':{}}
    particle['Field']=f
    particle['Definition']='EWSB'
    particle['Block']='DiracSpinors'
    particle['Properties']['Lorentz']='DiracSpinor'
    if type(dict_of_spinors[f])==list:
        particle['Parents']=dict_of_spinors[f][0]
        Majorana=False
        if 0 in dict_of_spinors[f]:
            Majorana=True
        elif dict_of_spinors[f][1].find(  
             dict_of_spinors[f][0])>-1:
            Majorana=True
        if not Majorana:
            particle['Properties']['DiracSpinor']=dict_of_spinors[f]
        else:
            particle['Properties']['Lorentz']='MajoranaSpinor'
            particle['Properties']['MajoranaSpinor']=rotations['EWSB']['DiracSpinors'][f]
    
    return particle


#Parameter defintions
def get_hypercharge(field,particles):
    try: 
        Y=particles[particles['Field']==field].reset_index(
              ).loc[0].get('Properties').get('Groups'
                                   )[0]
    except KeyError:
        Y=None
    return Y

def get_Lorentz(field,particles):
    try: 
        Y=particles[particles['Field']==field].reset_index(
              ).loc[0].get('Properties').get('Lorentz'
                                   )
    except KeyError:
        Y=None
    return Y


def get_higgs_vev(H,particles):
    '''
    Get the vev associated to the Yukawa coupling
    in `smdict` with Description `k`
    '''
    if H:
        H0=get_H0(H,particles)
        hh=get_hh(H0,particles)
        if not hh.empty:
            v=hh.get('Properties').apply(lambda d: d.get('vev')).loc[0]
    else:
        v=''
    return v

def get_diagonal_basis(vev,p,particles):
    try:
        pp=particles[particles['Parents']==p].reset_index(drop=True).loc[0,'Field']
        DF=particles[particles['Parents']==pp].reset_index(drop=True).loc[0,'Field']
    except:
        DF=''
    if vev and DF:
        db=r'''Sqrt[2]/%s* {{Mass[%s,1],0,0 },
                {0, Mass[%s,2],0},
                {0, 0, Mass[%s,3]}}''' %(vev,DF,DF,DF)
    else:
        db=''
    return db

def get_multiplet(WF,particles):
    '''WF: Weyl Fermion'''
    mltp=particles[particles['Parents']==WF]
    if mltp.shape[0]==2:
        chiral='Left'
    elif mltp.shape[0]==1:
        chiral='Right'
    else:
        chiral=None
    j=0
    multiplet={}
    for p in mltp['Field']:
        multiplet[p]={}
        j=j+1
        multiplet[p]['chiral']=chiral
        if chiral=='Left':
            multiplet[p]['dim']='doublet'
            if j==1:
                multiplet[p]['pos']='Up'
            elif j==2:
                multiplet[p]['pos']='Down'
            else:
                multiplet[p]['pos']=None
        else:
            multiplet[p]['dim']='singlet'
    return multiplet

def sorted_equality(l1,l2):
    return sorted(l1)==sorted(l2)

def get_H0(H,particles):
    "H is string"
    Hs=particles[particles['Parents']==H]
    #Get doublet components
    if Hs.shape[0]==2:
        # extract neutral component
        Hs=particles[particles['Parents']==H]
        H0s=Hs[Hs.get('Properties').apply(lambda d: d.get('ElectricCharge'))==0].reset_index(drop=True)
        if not H0s.get('Properties').empty:
            return H0s
def get_hh(H0,particles):
    "H0 is a dataframe"
    hh=particles[particles.get('Parents')==H0.loc[0,'Field']]
    if not hh.empty:
        return hh[hh.get('Properties').apply(lambda d: d.get('CP')=='Real')].reset_index(drop=True)
    else:
        return pd.DataFrame()            

#SPHENO Definitions

def get_tadpoles_and_bilinears(L,dimL,smd,exclude=None):
    #Get tadpoles from Lagrangina excluding bilinear `exclude`
    tadpoles=list(L[(np.logical_and(L==1,dimL==2))].index)
    ctdpl=[] # Parameters to be calculated from tadpoles
    cbln =[] # Bilinear input paramater
    for t in tadpoles:
        bln=smd.loc['Coupling',t]
        if bln!=exclude:
            ctdpl.append( bln )
        else:
            cbln.append(bln)
    return ctdpl,cbln

def get_input_parameters(L,dimL,smd):
    #TODO: Include Yukawas
    sci=[]
    scalarint=list(L[(np.logical_and(L==1,dimL>2))].index)
    for sc in scalarint:
        sci.append( smd.loc['Coupling',sc] )
    return sci

#TODO: Incluede Input Yukawas
def get_input_parameters_IN(sci,suffix='IN'):
    sciIN=[]
    BLS=[]
    for p in sci:
        rp=re.search('(\w+)',p)
        if rp:
            sciIN.append( rp.groups()[0]+suffix )

    if len(sci)==len(sciIN):
        return sciIN
    else:
        sys.exit('ERROR: Input parameter mismatch {}!={}'.format(sci,sciIN))
        
def get_BoundaryLowScaleInput(sci,sciIN):
    BLS=[]
    for i in range(len(sci)):
        BLS.append([sci[i],sciIN[i]])
    return BLS
def get_MINPAR(sciIN):
    MP=[]
    for i in range(len(sciIN)):
        MP.append([i+1,sciIN[i]])
    return MP

def get_smyukawas(smd,def_smyukawas=
                      ['Down-Yukawa-Coupling', 'Lepton-Yukawa-Coupling', 'Up-Yukawa-Coupling']):
    smyc=[]
    for y in def_smyukawas:
        smyc.append( smd.loc['Coupling',y])
    return smyc

def get_other_yukawas(smd,L,dimL,def_smyukawas=
                      ['Down-Yukawa-Coupling', 'Lepton-Yukawa-Coupling', 'Up-Yukawa-Coupling']):
    def_yukawas=list(L[(np.logical_and(L==2,dimL==3))].index)
    oyc=[]
    for y in def_yukawas:
        if y not in def_smyukawas:
            oyc.append( smd.loc['Coupling',y])
    return oyc

def get_gauge_couplings(smc,smcouplings=['Hypercharge-Coupling','Left-Coupling','Strong-Coupling']):
    cs=[]
    for c in smcouplings:
        cs.append( smc.loc['Description'][ smc.loc['Description']==c ].index[0] )
    
    return cs

def get_SM_MatchingConditions(smd,smc,
                              smcouplings=['Hypercharge-Coupling','Left-Coupling','Strong-Coupling'],
                              def_smyukawas=
                              ['Down-Yukawa-Coupling', 'Lepton-Yukawa-Coupling', 'Up-Yukawa-Coupling']):
    #VEV
    smvev=smd.loc['Coupling','EW-VEV']
    #Gauge couplings
    cs=get_gauge_couplings(smc,smcouplings)
    #SM Yukawas
    smyc=get_smyukawas(smd,def_smyukawas)

    lsmcpl=[]
    smcpl =[smvev]+smyc+cs
    for c in smcpl:
        lsmcpl.append( [c,c+'SM'])
    return lsmcpl

def get_decay_particles(DecayParticles   = ['Fu', 'Fe', 'Fd', 'hh'],
                    DecayParticles3B = ['Fu', 'Fe', 'Fd']):
    LDecayParticles3B=[]
    for p in DecayParticles3B:
        LDecayParticles3B.append( [p,'"{}.f90"'.format(p)])
    return DecayParticles,LDecayParticles3B


def get_sm_DefaultInputValues(smd,sci,sciIN):
    for i in range(len(sci)):
        if smd.loc['Coupling','SM Higgs Selfcouplings']==sci[i]:
            d={sciIN[i]:0.27}
        else:
            d={}
        return d
    
def to_math_list( l ):
    import re
    sl=str(l).replace('[','{' ).replace(
                      ']','}'     ).replace(
                       "'",""   ).replace(
                       r'\\','\\')
    sl=re.sub(r'\\{(\w+)}',r'\\[\1]',sl)
    return sl

def to_SPheno(SP,file,dictentries=['DefaultInputValues']):
    '''dicentries:  dictionaries to be printed directly'''
    f=open(file,'w')
    for i in SP.index:
        if type(SP.loc[i,'Properties'])==bool:
            f.write('{} = {};\n\n'.format(i,SP.loc[i,'Properties']))
        elif type(SP.loc[i,'Properties'])==list:
            f.write('{} = {};\n\n'.format(i,to_math_list(SP.loc[i,'Properties'])  ))
        elif type(SP.loc[i,'Properties'])==dict:
            d=SP.loc[i,'Properties']
            if i in dictentries:
                dfv='{} = {{'.format(i)
                for k in d.keys():
                    dfv=dfv+'{} -> {}'.format(k,0.27) 
                dfv=dfv+'};\n\n'
                f.write(dfv)
            else:
                for k in d.keys():
                    if k  in ['MatchingConditions']:
                        f.write('{}[{}]={};\n\n'.format(i,k,to_math_list( SP.loc[i,'Properties'][k] )    
                              ))
    f.close()

def to_defintions(dpp,symbol='Field'):
    dppc=copy.deepcopy(dpp)
    PPDefinitions={}
    for k  in list(set( [d.get('Definition') for d in dppc] )):
        if k==None:
            PPDefinitions.update({'Properties':{}})
            [d.update({'Definition':'Properties'}) for d in dppc]
        else:
            PPDefinitions.update({k:{}})
            
    for d in dppc:
        #print(d.get(symbol))
        PPDefinitions[d.get('Definition')].update(
                                          {d.get(symbol): {}}
                                                       )
        if d.get('Description'):
            PPDefinitions[d.get('Definition')][d.get(symbol)].update(
                   {'Description':d.get('Description')})
        ud=d.get('Properties').get('update_Description')
        if ud:
            PPDefinitions[d.get('Definition')][d.get(symbol)].update(
                  ud )
    return PPDefinitions    

Overwriting SARAH.py


## 2) Get parameters from `DEFINITION[GaugeES]`

In [8]:
dsbd=parse_mathematica_list_of_list(smf,NAME='DEFINITION',KEY='GaugeES')

fdotm=smf
d={}
for lag in dsbd['LagrangianInput']:
    pattern='^\s*\t*({}.*)'.format(lag[0])
    pattern_start='='
    pattern_end=';'
    only_extract=True
    l=extract_code_block(fdotm,pattern,pattern_start,pattern_end,only_extract)
    fl=list(l.values())[0]

    fll=re.sub('[\+\-]','::',
        re.sub('\)*\s*\t*\;\s*\t*$','',
            re.sub( '^\s*\t*[\-\+\(]+','',
            fl))).split('::')

    for x in fll:
        flli=re.sub('^\s*\t*', '',x)
        dd=re.sub('([\w\\\/\s\[\]]+)\s+([\w\[\]\.]+)',r'\1::\2',  flli).split('::')
        d[dd[0].strip()]=dd[1].strip()

In [9]:
d

{'1/2 \\[Lambda]': 'conj[H].H.conj[H].H',
 'Yd': 'conj[H].d.q',
 'Ye': 'conj[H].e.l',
 'Yu': 'u.q.H',
 'mu2': 'conj[H].H'}

In [10]:
#particles=pd.read_json('parseparticles.json')
particles=pd.read_json('JSON/fullparticles.json')

In [11]:
particles

,Block,Definition,Description,Field,Parents,Properties,rotation
0,None,GaugeES,B-Boson,VB,None,"{'Index': ' hypercharge', 'SSB': ' g1', 'Group': ' U[1]', 'Lorentz': 'Vector', 'Coupling': ' g1'}",None
1,NaN,GaugeES,B-Boson Ghost,gB,VB,"{'Group': ' U[1]', 'Lorentz': 'Scalar'}",NaN
2,None,GaugeES,W-Bosons,VWB,None,"{'Index': ' left', 'SSB': ' g2', 'Group': ' SU[2]', 'Lorentz': 'Vector', 'Coupling': ' g2'}",None
3,NaN,GaugeES,W-Boson Ghost,gWB,VWB,"{'Group': ' SU[2]', 'Lorentz': 'Scalar'}",NaN
4,None,GaugeES,Gluon,VG,None,"{'Index': ' color', 'SSB': ' g3', 'Group': ' SU[3]', 'Lorentz': 'Vector', 'Coupling': ' g3'}",None
5,NaN,GaugeES,Gluon Ghost,gG,VG,"{'Group': ' SU[3]', 'Lorentz': 'Scalar'}",NaN
6,None,WeylFermionAndIndermediate,NaN,q,None,"{'update_Description': {'LaTeX': 'q'}, 'Groups': ['1/6', '2', '3'], 'NF': '3', 'multiplet': ['uL', 'dL'], 'Lorentz': 'WeylFermion'}",None
7,None,WeylFermionAndIndermediate,NaN,l,None,"{'update_Description': {'LaTeX': 'l'}, 'Groups': ['-1/2', '2', '1'], 'NF': '3', 'multiplet': ['vL', 'eL'], 'Lorentz': 'WeylFermion'}",None
8,None,WeylFermionAndIndermediate,NaN,d,None,"{'update_Description': {'LaTeX': 'd'}, 'Groups': ['1/3', '1', '-3'], 'NF': '3', 'Lorentz': 'WeylFermion'}",None
9,None,WeylFermionAndIndermediate,NaN,u,None,"{'update_Description': {'LaTeX': 'u'}, 'Groups': ['-2/3', '1', '-3'], 'NF': '3', 'Lorentz': 'WeylFermion'}",None


In [12]:
dd={}
for k in d.keys():
    prtcls=[ s.strip() for s in re.sub( '[\w]+\[(\w+)\]',r'\1',d[k]).split('.')]
    print(prtcls)
    dd[k]={'operator':d[k],'fields':prtcls,
           'hypercharge': sorted( [ get_hypercharge(f,particles).replace('-','') 
                                 for f in prtcls]),
           'Lorentz':[ get_Lorentz(f,particles) for f in prtcls]
          }
    
for k in dd.keys():
    ck=re.sub('^[0-9\/\s]+',r'',k)
    if ck!=k:
        vk=dd.pop(k)
        dd[ck]=vk    

['u', 'q', 'H']
['H', 'e', 'l']
['H', 'H']
['H', 'd', 'q']
['H', 'H', 'H', 'H']


In [13]:
dd

{'Yd': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'd', 'q'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'operator': 'conj[H].d.q'},
 'Ye': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'e', 'l'],
  'hypercharge': ['1', '1/2', '1/2'],
  'operator': 'conj[H].e.l'},
 'Yu': {'Lorentz': ['WeylFermion', 'WeylFermion', 'Scalar'],
  'fields': ['u', 'q', 'H'],
  'hypercharge': ['1/2', '1/6', '2/3'],
  'operator': 'u.q.H'},
 '\\[Lambda]': {'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['H', 'H', 'H', 'H'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'operator': 'conj[H].H.conj[H].H'},
 'mu2': {'Lorentz': ['Scalar', 'Scalar'],
  'fields': ['H', 'H'],
  'hypercharge': ['1/2', '1/2'],
  'operator': 'conj[H].H'}}

## Extract the SM coupling from Model file
Check the Yukawa interactions to extract the SM Yukawa couplings, and identify the Higgs field to get the scalar couplings

In [14]:
smdict={'Down-Yukawa-Coupling':{'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
                                'hypercharge': ['1/2', '1/3', '1/6'],
                                'update_Description':{}},
        'Up-Yukawa-Coupling':{'Lorentz': ['WeylFermion', 'WeylFermion','Scalar'],
                              'hypercharge': ['1/2', '1/6', '2/3'],
                             'update_Description':{}},
        'Lepton-Yukawa-Coupling':{'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
                                  'hypercharge': ['1', '1/2', '1/2'],
                                 'update_Description':{}},
        'SM Mu Parameter':  {'Lorentz': ['Scalar', 'Scalar'],
                             'hypercharge': ['1/2', '1/2'],
                             'update_Description':{'OutputName':'m2SM'}},
  'SM Higgs Selfcouplings': {'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
                             'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
                            'update_Description':{}}
       }

In [15]:
smdict

{'Down-Yukawa-Coupling': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {}},
 'Lepton-Yukawa-Coupling': {'Lorentz': ['Scalar',
   'WeylFermion',
   'WeylFermion'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {}},
 'SM Higgs Selfcouplings': {'Lorentz': ['Scalar',
   'Scalar',
   'Scalar',
   'Scalar'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Lorentz': ['Scalar', 'Scalar'],
  'hypercharge': ['1/2', '1/2'],
  'update_Description': {'OutputName': 'm2SM'}},
 'Up-Yukawa-Coupling': {'Lorentz': ['WeylFermion', 'WeylFermion', 'Scalar'],
  'hypercharge': ['1/2', '1/6', '2/3'],
  'update_Description': {}}}

In [16]:
# Get standard model Higgs
import sys
for k in dd.keys():
    if (sorted_equality( smdict['Up-Yukawa-Coupling']['Lorentz'],dd[k]['Lorentz'] ) and
        sorted_equality( smdict['Up-Yukawa-Coupling']['hypercharge'],dd[k]['hypercharge'] ) ):
        #Get Higgs from scalar part
        for i in range(len(dd[k]['Lorentz'])):
            if dd[k]['Lorentz'][i]=='Scalar':
                H=dd[k]['fields'][i]
                smdict['Up-Yukawa-Coupling']['Coupling']=k
                smdict['Up-Yukawa-Coupling']['Higgs']=H
                smdict['Up-Yukawa-Coupling']['fields']=dd[k]['fields']
        #Use obtained Higgs to obtain diagonal form
        if not H:
            sys.exit('Higgs doublet field symbol not found!')
        for i in range(len(dd[k]['Lorentz'])):                
            if dd[k]['Lorentz'][i]=='WeylFermion':
                mltp=get_multiplet(dd[k]['fields'][i],particles)
                for p in mltp.keys():
                    if mltp[p].get('pos')=='Up':
                        vev=get_higgs_vev(H,particles)
                        smdict['Up-Yukawa-Coupling'
                              ]['update_Description'
                              ]['DependenceNum']=get_diagonal_basis(
                                                 vev,p,particles)                
                
if not smdict.get('Up-Yukawa-Coupling'):
    sys.exit('"Up-Yukawa-Coupling" NOT FOUND!' )
lk=list(dd.keys())
try:
    lk.remove( smdict['Up-Yukawa-Coupling']['Coupling'] )
except KeyError:
    pass

lds=list(smdict.keys())
lds.remove('Up-Yukawa-Coupling')
for ds in lds:
    for k in lk:
        if (sorted_equality( smdict[ds]['Lorentz'],dd[k]['Lorentz'] ) and
            sorted_equality( smdict[ds]['hypercharge'],dd[k]['hypercharge'] ) ):
            smdict[ds]['Coupling']=k
            smdict[ds]['fields']=dd[k]['fields']
            if H:
                smdict[ds]['Higgs']=H
            for i in range(len(dd[k]['Lorentz'])):
                if dd[k]['Lorentz'][i]=='WeylFermion':
                    mltp=get_multiplet(dd[k]['fields'][i],particles)
                    for p in mltp.keys():
                        if mltp[p].get('pos')=='Down':
                            print(p,mltp[p].get('pos'))
                            Hk=[dd[k].get('fields')[i] for i in range(len( dd[k]['Lorentz'] ))
                                 if dd[k]['Lorentz'][i]=='Scalar' ][0]
                            vev=get_higgs_vev(Hk,particles)
                            smdict[ds]['update_Description'
                                  ]['DependenceNum']=get_diagonal_basis(
                                                     vev,p,particles)                    

dL Down
eL Down


In [17]:
smdict

{'Down-Yukawa-Coupling': {'Coupling': 'Yd',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'd', 'q'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}},
 'Lepton-Yukawa-Coupling': {'Coupling': 'Ye',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'e', 'l'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}},
 'SM Higgs Selfcouplings': {'Coupling': '\\[Lambda]',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['H', 'H', 'H', 'H'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Coupling': 'mu2',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar'],
  'f

In [18]:
particles

,Block,Definition,Description,Field,Parents,Properties,rotation
0,None,GaugeES,B-Boson,VB,None,"{'Index': ' hypercharge', 'SSB': ' g1', 'Group': ' U[1]', 'Lorentz': 'Vector', 'Coupling': ' g1'}",None
1,NaN,GaugeES,B-Boson Ghost,gB,VB,"{'Group': ' U[1]', 'Lorentz': 'Scalar'}",NaN
2,None,GaugeES,W-Bosons,VWB,None,"{'Index': ' left', 'SSB': ' g2', 'Group': ' SU[2]', 'Lorentz': 'Vector', 'Coupling': ' g2'}",None
3,NaN,GaugeES,W-Boson Ghost,gWB,VWB,"{'Group': ' SU[2]', 'Lorentz': 'Scalar'}",NaN
4,None,GaugeES,Gluon,VG,None,"{'Index': ' color', 'SSB': ' g3', 'Group': ' SU[3]', 'Lorentz': 'Vector', 'Coupling': ' g3'}",None
5,NaN,GaugeES,Gluon Ghost,gG,VG,"{'Group': ' SU[3]', 'Lorentz': 'Scalar'}",NaN
6,None,WeylFermionAndIndermediate,NaN,q,None,"{'update_Description': {'LaTeX': 'q'}, 'Groups': ['1/6', '2', '3'], 'NF': '3', 'multiplet': ['uL', 'dL'], 'Lorentz': 'WeylFermion'}",None
7,None,WeylFermionAndIndermediate,NaN,l,None,"{'update_Description': {'LaTeX': 'l'}, 'Groups': ['-1/2', '2', '1'], 'NF': '3', 'multiplet': ['vL', 'eL'], 'Lorentz': 'WeylFermion'}",None
8,None,WeylFermionAndIndermediate,NaN,d,None,"{'update_Description': {'LaTeX': 'd'}, 'Groups': ['1/3', '1', '-3'], 'NF': '3', 'Lorentz': 'WeylFermion'}",None
9,None,WeylFermionAndIndermediate,NaN,u,None,"{'update_Description': {'LaTeX': 'u'}, 'Groups': ['-2/3', '1', '-3'], 'NF': '3', 'Lorentz': 'WeylFermion'}",None


In [19]:
rotation={}
for k in smdict.keys():
    print(k)
    for i in range(len(smdict[k].get('fields'))):
        if smdict[k]['Lorentz'][i]=='WeylFermion':
            mltp=particles[particles['Parents']==smdict[k]['fields'][i]]
            if mltp.shape[0]==2:
                chiral='Left'
            elif mltp.shape[0]==1:
                chiral='Right'
            else:
                chiral=None
                
            j=0
            for p in mltp['Field']:
                j=j+1
                print(p)
                #Updage smdict
                #rotations
                prt=particles.dropna(subset=['Parents'])
                rot=prt[prt['Parents'].dropna(
                       ).str.contains(p)].reset_index()['rotation'].loc[0]
                if type(rot)==str:
                    rotation[p]={rot:{}}
                    dscr='Mixing-Matrix'
                    if chiral=='Left':
                        if j==1:
                            dscr='{}-Up-{}'.format(chiral,dscr)
                        elif j==2:
                            if re.search('^[dD]',p):
                                dscr='{}-Down-{}'.format(chiral,dscr)
                            elif re.search('^[eE]',p):
                                dscr='{}-Lepton-{}'.format(chiral,dscr)
            
        
                    elif chiral=='Right':
                        if re.search('^[uU]',p):
                            dscr='{}-Up-{}'.format(chiral,dscr)
                        elif re.search('^[dD]',p):
                            dscr='{}-Down-{}'.format(chiral,dscr)
                        elif re.search('^[eE]',p):
                            dscr='{}-Lepton-{}'.format(chiral,dscr)
                        else:
                            dscr=None
                    rotation[p][rot]['Description']=dscr

SM Higgs Selfcouplings
Down-Yukawa-Coupling
dR
uL
dL
Lepton-Yukawa-Coupling
eR
vL
eL
Up-Yukawa-Coupling
uR
uL
dL
SM Mu Parameter


In [20]:
smdict

{'Down-Yukawa-Coupling': {'Coupling': 'Yd',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'd', 'q'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}},
 'Lepton-Yukawa-Coupling': {'Coupling': 'Ye',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'e', 'l'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}},
 'SM Higgs Selfcouplings': {'Coupling': '\\[Lambda]',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['H', 'H', 'H', 'H'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Coupling': 'mu2',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar'],
  'f

In [21]:
rotation

{'dL': {'Vd': {'Description': 'Left-Down-Mixing-Matrix'}},
 'dR': {'Ud': {'Description': 'Right-Down-Mixing-Matrix'}},
 'eL': {'Ve': {'Description': 'Left-Lepton-Mixing-Matrix'}},
 'eR': {'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'}},
 'uL': {'Vu': {'Description': 'Left-Up-Mixing-Matrix'}},
 'uR': {'Uu': {'Description': 'Right-Up-Mixing-Matrix'}}}

In [22]:
prtng=particles[~particles['Description'].astype(str).str.lower().str.contains('ghost')].reset_index(drop=True)

In [23]:
grps=prtng[prtng['Definition']=='GaugeES']['Properties'].apply(lambda d: d.get('Group')).dropna()

G={}
coupling={}
for g in grps:
    G=prtng[prtng['Properties'].apply(lambda d: d.get('Group')==g)].reset_index(drop=True)
    V=G.reset_index().loc[0,'Field']
    print('*',V)
    try:
        VV=prtng[prtng['Parents'].astype(str).str.contains(V)
                    ].reset_index(drop=True)
        if VV.shape[0]==2:
            VV=VV[VV.get('Properties').astype(str).str.contains(
                   'conj\[\w+\]')].reset_index(drop=True)
    
        f=VV.loc[0,'Field']
        r=VV.loc[0,'rotation']
    except KeyError:
        f=''    
    c=G.loc[0,'Properties']['Coupling'].strip()
    if g.find('U[1]')>-1:
        if f:
            rotation[f]={r: {'Description':"Photon-Z Mixing Matrix"}}
        coupling[c]={'Description':'Hypercharge-Coupling'}
    if g.find('SU[2]')>-1:
        print(g)
        if f:
            rotation[f]={r: {'Description':"W Mixing Matrix",
                              'update_Description':
                                    {'Dependence' :r'''1/Sqrt[2] {{1, 1},
                              {\[ImaginaryI],-\[ImaginaryI]}}''' }}}
        coupling[c]={'Description':'Left-Coupling'}
    if g.find('SU[3]')>-1:
        coupling[c]={'Description':'Strong-Coupling'}

    #break
    print(g)

* VB
   U[1]
* VWB
 SU[2]
 SU[2]
* VG
  SU[3]


In [24]:
rotation

{'VP': {'ZZ': {'Description': 'Photon-Z Mixing Matrix'}},
 'VWp': {'ZW': {'Description': 'W Mixing Matrix',
   'update_Description': {'Dependence': '1/Sqrt[2] {{1, 1},\n                              {\\[ImaginaryI],-\\[ImaginaryI]}}'}}},
 'dL': {'Vd': {'Description': 'Left-Down-Mixing-Matrix'}},
 'dR': {'Ud': {'Description': 'Right-Down-Mixing-Matrix'}},
 'eL': {'Ve': {'Description': 'Left-Lepton-Mixing-Matrix'}},
 'eR': {'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'}},
 'uL': {'Vu': {'Description': 'Left-Up-Mixing-Matrix'}},
 'uR': {'Uu': {'Description': 'Right-Up-Mixing-Matrix'}}}

In [25]:
particles

,Block,Definition,Description,Field,Parents,Properties,rotation
0,None,GaugeES,B-Boson,VB,None,"{'Index': ' hypercharge', 'SSB': ' g1', 'Group': ' U[1]', 'Lorentz': 'Vector', 'Coupling': ' g1'}",None
1,NaN,GaugeES,B-Boson Ghost,gB,VB,"{'Group': ' U[1]', 'Lorentz': 'Scalar'}",NaN
2,None,GaugeES,W-Bosons,VWB,None,"{'Index': ' left', 'SSB': ' g2', 'Group': ' SU[2]', 'Lorentz': 'Vector', 'Coupling': ' g2'}",None
3,NaN,GaugeES,W-Boson Ghost,gWB,VWB,"{'Group': ' SU[2]', 'Lorentz': 'Scalar'}",NaN
4,None,GaugeES,Gluon,VG,None,"{'Index': ' color', 'SSB': ' g3', 'Group': ' SU[3]', 'Lorentz': 'Vector', 'Coupling': ' g3'}",None
5,NaN,GaugeES,Gluon Ghost,gG,VG,"{'Group': ' SU[3]', 'Lorentz': 'Scalar'}",NaN
6,None,WeylFermionAndIndermediate,NaN,q,None,"{'update_Description': {'LaTeX': 'q'}, 'Groups': ['1/6', '2', '3'], 'NF': '3', 'multiplet': ['uL', 'dL'], 'Lorentz': 'WeylFermion'}",None
7,None,WeylFermionAndIndermediate,NaN,l,None,"{'update_Description': {'LaTeX': 'l'}, 'Groups': ['-1/2', '2', '1'], 'NF': '3', 'multiplet': ['vL', 'eL'], 'Lorentz': 'WeylFermion'}",None
8,None,WeylFermionAndIndermediate,NaN,d,None,"{'update_Description': {'LaTeX': 'd'}, 'Groups': ['1/3', '1', '-3'], 'NF': '3', 'Lorentz': 'WeylFermion'}",None
9,None,WeylFermionAndIndermediate,NaN,u,None,"{'update_Description': {'LaTeX': 'u'}, 'Groups': ['-2/3', '1', '-3'], 'NF': '3', 'Lorentz': 'WeylFermion'}",None


{\[Lambda],  { Description -> "SM Higgs Selfcouplings",
               DependenceNum -> Mass[hh]^2/(v^2)}},
{v,          { Description -> "EW-VEV",
               DependenceNum -> Sqrt[4*Mass[VWp]^2/(g2^2)],
               DependenceSPheno -> None,
               OutputName -> vvSM}},
{mH2,        { Description -> "SM Higgs Mass Parameter"}},

{ThetaW,    { Description -> "Weinberg-Angle",
              DependenceNum -> ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]  }},

In [27]:
H=smdict['SM Higgs Selfcouplings'].get('Higgs')
if H:
    H0=get_H0(H,particles)
    if not H0.empty:
        hh=get_hh(H0,particles).get('Field').loc[0]
        vev=get_hh(H0,particles).get('Properties').loc[0].get('vev')
        if hh and vev:
            smdict['SM Higgs Selfcouplings'
                  ]['update_Description']={
                    'DependenceNum': r'Mass[{}]^2/({}^2)'.format(hh,vev) }

In [28]:
smdict['EW-VEV']={}
smdict['EW-VEV']['Coupling']=vev
smdict['EW-VEV']['update_Description']={}
smdict['EW-VEV']['update_Description']['DependenceSPheno']=None
smdict['EW-VEV']['update_Description']['OutputName']='vvSM'
VWp=''
g2=''
for k in rotation.keys():
    if list( rotation[k].values() )[0].get('Description')=='W Mixing Matrix':
        VWp=k
for c in coupling.keys():
    if coupling[c].get('Description')=='Left-Coupling':
        g2=c
smdict['EW-VEV']['update_Description']['DependenceNum']=r'Sqrt[4*Mass[{}]^2/({}^2)]'.format(VWp,g2)

In [29]:
for k in rotation:
    if list( rotation[k].values() )[0].get('Description')=='Photon-Z Mixing Matrix':
        VP=k
        ZZ=list(rotation[k].keys())[0]
fz=particles[particles['rotation']==ZZ]
VZ=fz[fz.get('Field')!=VP].reset_index(drop=True).loc[0,'Field']
smdict['Weinberg-Angle']={}
smdict['Weinberg-Angle']['Coupling']='ThetaW'
smdict['Weinberg-Angle']['update_Description']={}
smdict['Weinberg-Angle']['update_Description'
     ]['DependenceNum']='ArcSin[Sqrt[1 - Mass[{}]^2/Mass[{}]^2]]'.format(
                                                           VWp,VZ )

In [30]:
constants={}
constants['AlphaS']= { 'Description'  : 'Alpha Strong'}
constants['e']     = { 'Description'  :  'electric charge'} 
constants['Gf']    = { 'Description'  :  "Fermi's constant"}
constants['aEWinv']= { 'Description'  :  'inverse weak coupling constant at mZ'}
constants['mH2']   = { 'Description'  :  'SM Higgs Mass Parameter'}

In [31]:
smdict,coupling,rotation,constants

({'Down-Yukawa-Coupling': {'Coupling': 'Yd',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
   'fields': ['H', 'd', 'q'],
   'hypercharge': ['1/2', '1/3', '1/6'],
   'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}},
  'EW-VEV': {'Coupling': 'v',
   'update_Description': {'DependenceNum': 'Sqrt[4*Mass[VWp]^2/(g2^2)]',
    'DependenceSPheno': None,
    'OutputName': 'vvSM'}},
  'Lepton-Yukawa-Coupling': {'Coupling': 'Ye',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
   'fields': ['H', 'e', 'l'],
   'hypercharge': ['1', '1/2', '1/2'],
   'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}},
  'SM Higgs Selfcouplings': {'Coupling': '\\[Lambda]',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
   'fields': ['H', 'H', 'H', 

In [32]:
parameters=[]

for k in smdict:
    d={}
    d['Description']=k
    d['Name']=d['Description']
    d['Properties']=smdict[k]
    d['Symbol']=d.get('Properties').get('Coupling')
    d['Class']='Lagrangian'
    kk=parameters.append(d)

In [33]:
import copy
#rotation will be altered
rt=copy.deepcopy(rotation)
for k in rt.keys():
    for r in rt[k].keys():
        d={}
        d['Symbol']=r
        d['Description']=rt[k][r].get('Description')
        d['Name']=d['Description']
        d['Class']='Rotation'
        kk=rt[k][r].pop('Description')
        if rt[k][r]:
            d['Properties']=rt[k][r]
        else:
            d['Properties']={}
        parameters.append(d)
del(rt)        

In [34]:
for k in coupling.keys():
    d={}
    d['Symbol']=k
    d['Description']=coupling[k].get('Description')
    d['Name']=d['Description']
    d['Class']='Coupling'
    d['Properties']={}
    parameters.append(d)

In [35]:
for k in constants.keys():
    d={}
    d['Symbol']=k
    d['Description']=constants[k].get('Description')
    d['Name']=d['Description']
    d['Class']='Constant'
    d['Properties']={}
    parameters.append(d)

In [36]:
pd.DataFrame(parameters)

,Class,Description,Name,Properties,Symbol
0,Lagrangian,Down-Yukawa-Coupling,Down-Yukawa-Coupling,"{'fields': ['H', 'd', 'q'], 'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 }, {0, Mass[Fd,2],0}, {0, 0, Mass[Fd,3]}}'}, 'hypercharge': ['1/2',...",Yd
1,Lagrangian,Lepton-Yukawa-Coupling,Lepton-Yukawa-Coupling,"{'fields': ['H', 'e', 'l'], 'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 }, {0, Mass[Fe,2],0}, {0, 0, Mass[Fe,3]}}'}, 'hypercharge': ['1', '...",Ye
2,Lagrangian,SM Mu Parameter,SM Mu Parameter,"{'fields': ['H', 'H'], 'update_Description': {'OutputName': 'm2SM'}, 'hypercharge': ['1/2', '1/2'], 'Lorentz': ['Scalar', 'Scalar'], 'Higgs': 'H', 'Coupling': 'mu2'}",mu2
3,Lagrangian,EW-VEV,EW-VEV,"{'update_Description': {'OutputName': 'vvSM', 'DependenceSPheno': None, 'DependenceNum': 'Sqrt[4*Mass[VWp]^2/(g2^2)]'}, 'Coupling': 'v'}",v
4,Lagrangian,SM Higgs Selfcouplings,SM Higgs Selfcouplings,"{'fields': ['H', 'H', 'H', 'H'], 'update_Description': {'DependenceNum': 'Mass[hh]^2/(v^2)'}, 'hypercharge': ['1/2', '1/2', '1/2', '1/2'], 'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'], 'Hig...",\[Lambda]
5,Lagrangian,Weinberg-Angle,Weinberg-Angle,"{'update_Description': {'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]'}, 'Coupling': 'ThetaW'}",ThetaW
6,Lagrangian,Up-Yukawa-Coupling,Up-Yukawa-Coupling,"{'fields': ['u', 'q', 'H'], 'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fu,1],0,0 }, {0, Mass[Fu,2],0}, {0, 0, Mass[Fu,3]}}'}, 'hypercharge': ['1/2',...",Yu
7,Rotation,Right-Up-Mixing-Matrix,Right-Up-Mixing-Matrix,{},Uu
8,Rotation,Left-Lepton-Mixing-Matrix,Left-Lepton-Mixing-Matrix,{},Ve
9,Rotation,Right-Lepton-Mixing-Matrix,Right-Lepton-Mixing-Matrix,{},Ue


In [37]:
parameters

[{'Class': 'Lagrangian',
  'Description': 'Down-Yukawa-Coupling',
  'Name': 'Down-Yukawa-Coupling',
  'Properties': {'Coupling': 'Yd',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
   'fields': ['H', 'd', 'q'],
   'hypercharge': ['1/2', '1/3', '1/6'],
   'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}},
  'Symbol': 'Yd'},
 {'Class': 'Lagrangian',
  'Description': 'Lepton-Yukawa-Coupling',
  'Name': 'Lepton-Yukawa-Coupling',
  'Properties': {'Coupling': 'Ye',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
   'fields': ['H', 'e', 'l'],
   'hypercharge': ['1', '1/2', '1/2'],
   'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}},
  'Symbol': 'Ye'},
 {'Class': 'Lagrangian',
  'Description': 'SM Mu Parameter',
  'Name': 'SM Mu Parameter',
  'Prope

In [38]:
rotation

{'VP': {'ZZ': {'Description': 'Photon-Z Mixing Matrix'}},
 'VWp': {'ZW': {'Description': 'W Mixing Matrix',
   'update_Description': {'Dependence': '1/Sqrt[2] {{1, 1},\n                              {\\[ImaginaryI],-\\[ImaginaryI]}}'}}},
 'dL': {'Vd': {'Description': 'Left-Down-Mixing-Matrix'}},
 'dR': {'Ud': {'Description': 'Right-Down-Mixing-Matrix'}},
 'eL': {'Ve': {'Description': 'Left-Lepton-Mixing-Matrix'}},
 'eR': {'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'}},
 'uL': {'Vu': {'Description': 'Left-Up-Mixing-Matrix'}},
 'uR': {'Uu': {'Description': 'Right-Up-Mixing-Matrix'}}}

In [41]:
ParameterDefinitions=to_defintions(parameters,symbol='Symbol')
ParameterDefinitions

{'Properties': {'AlphaS': {'Description': 'Alpha Strong'},
  'Gf': {'Description': "Fermi's constant"},
  'ThetaW': {'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]',
   'Description': 'Weinberg-Angle'},
  'Ud': {'Description': 'Right-Down-Mixing-Matrix'},
  'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'},
  'Uu': {'Description': 'Right-Up-Mixing-Matrix'},
  'Vd': {'Description': 'Left-Down-Mixing-Matrix'},
  'Ve': {'Description': 'Left-Lepton-Mixing-Matrix'},
  'Vu': {'Description': 'Left-Up-Mixing-Matrix'},
  'Yd': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}',
   'Description': 'Down-Yukawa-Coupling'},
  'Ye': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}',
   'Description': 'Lepton-Yukawa-Coupling'},
  'Yu': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fu,1],0,0 },\n                {0, Mass[Fu,2],0},\n                {0, 0

In [48]:
pd.DataFrame(ParameterDefinitions)

,Properties
AlphaS,{'Description': 'Alpha Strong'}
Gf,{'Description': 'Fermi's constant'}
ThetaW,"{'Description': 'Weinberg-Angle', 'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]'}"
Ud,{'Description': 'Right-Down-Mixing-Matrix'}
Ue,{'Description': 'Right-Lepton-Mixing-Matrix'}
Uu,{'Description': 'Right-Up-Mixing-Matrix'}
Vd,{'Description': 'Left-Down-Mixing-Matrix'}
Ve,{'Description': 'Left-Lepton-Mixing-Matrix'}
Vu,{'Description': 'Left-Up-Mixing-Matrix'}
Yd,"{'Description': 'Down-Yukawa-Coupling', 'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 }, {0, Mass[Fd,2],0}, {0, 0, Mass[Fd,3]}}'}"


## Write file

In [42]:
f=open('SMp.json','w')
json.dump(ParameterDefinitions,f)
f.close()

In [43]:
CHECK_PANDAS_SAVED_JSON=True
if CHECK_PANDAS_SAVED_JSON:
    SM=pd.read_json('SMp.json')#,lines=True,orient='records')

In [44]:
SM.columns

Index(['Properties'], dtype='object')

In [45]:
SM.columns
c='Properties'
SM[c].dropna().index

Index(['AlphaS', 'Gf', 'ThetaW', 'Ud', 'Ue', 'Uu', 'Vd', 'Ve', 'Vu', 'Yd',
       'Ye', 'Yu', 'ZW', 'ZZ', '\[Lambda]', 'aEWinv', 'e', 'g1', 'g2', 'g3',
       'mH2', 'mu2', 'v'],
      dtype='object')

In [46]:
print( to_math(SM,'para.m',definitions='ParameterDefinitions') )

ParameterDefinitions = {
    {AlphaS, { Description -> "Alpha Strong"}},
    {Gf, { Description -> "Fermi's constant"}},
    {ThetaW, { Description -> "Weinberg-Angle",
           DependenceNum -> ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]}},
    {Ud, { Description -> "Right-Down-Mixing-Matrix"}},
    {Ue, { Description -> "Right-Lepton-Mixing-Matrix"}},
    {Uu, { Description -> "Right-Up-Mixing-Matrix"}},
    {Vd, { Description -> "Left-Down-Mixing-Matrix"}},
    {Ve, { Description -> "Left-Lepton-Mixing-Matrix"}},
    {Vu, { Description -> "Left-Up-Mixing-Matrix"}},
    {Yd, { Description -> "Down-Yukawa-Coupling",
           DependenceNum -> Sqrt[2]/v* {{Mass[Fd,1],0,0 },
                {0, Mass[Fd,2],0},
                {0, 0, Mass[Fd,3]}}}},
    {Ye, { Description -> "Lepton-Yukawa-Coupling",
           DependenceNum -> Sqrt[2]/v* {{Mass[Fe,1],0,0 },
                {0, Mass[Fe,2],0},
                {0, 0, Mass[Fe,3]}}}},
    {Yu, { Description -> "Up-Yukawa-Coupling",
          

In [47]:
cat para.m

ParameterDefinitions = {
    {AlphaS, { Description -> "Alpha Strong"}},
    {Gf, { Description -> "Fermi's constant"}},
    {ThetaW, { Description -> "Weinberg-Angle",
           DependenceNum -> ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]}},
    {Ud, { Description -> "Right-Down-Mixing-Matrix"}},
    {Ue, { Description -> "Right-Lepton-Mixing-Matrix"}},
    {Uu, { Description -> "Right-Up-Mixing-Matrix"}},
    {Vd, { Description -> "Left-Down-Mixing-Matrix"}},
    {Ve, { Description -> "Left-Lepton-Mixing-Matrix"}},
    {Vu, { Description -> "Left-Up-Mixing-Matrix"}},
    {Yd, { Description -> "Down-Yukawa-Coupling",
           DependenceNum -> Sqrt[2]/v* {{Mass[Fd,1],0,0 },
                {0, Mass[Fd,2],0},
                {0, 0, Mass[Fd,3]}}}},
    {Ye, { Description -> "Lepton-Yukawa-Coupling",
           DependenceNum -> Sqrt[2]/v* {{Mass[Fe,1],0,0 },
                {0, Mass[Fe,2],0},
                {0, 0, Mass[Fe,3]}}}},
    {Yu, { Description -> "Up-Yukawa-Coupling",
          

## OLD WAY: Build `parameters` with the same standards as `particles`
TODO: Switch to fullparameters.json

In [27]:
ParameterDefinitions={'Properties':{}}
ParameterDefinitions['Properties'].update(coupling)
ParameterDefinitions['Properties'].update(constants)
for k in rotation.keys():
    ParameterDefinitions['Properties'].update(rotation.get(k))

#ParameterDefinition

In [28]:
 list(smdict.keys())

['Lepton-Yukawa-Coupling',
 'SM Higgs Selfcouplings',
 'Up-Yukawa-Coupling',
 'SM Mu Parameter',
 'Down-Yukawa-Coupling',
 'Weinberg-Angle',
 'EW-VEV']

In [29]:
#k='Down-Yukawa-Coupling'
for k in smdict.keys():
    smp={}
    smp['Description']=k
    smp.update(smdict[k].get('update_Description'))
    ParameterDefinitions['Properties'].update( { smdict[k].get('Coupling'):  smp} )
    print("*"*30)
    print( { smdict[k].get('Coupling'):  smp} )
    print("*"*30)
    #break

******************************
{'Ye': {'Description': 'Lepton-Yukawa-Coupling', 'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}}
******************************
******************************
{'\\[Lambda]': {'Description': 'SM Higgs Selfcouplings', 'DependenceNum': 'Mass[hh]^2/(v^2)'}}
******************************
******************************
{'Yu': {'Description': 'Up-Yukawa-Coupling', 'DependenceNum': 'Sqrt[2]/v* {{Mass[Fu,1],0,0 },\n                {0, Mass[Fu,2],0},\n                {0, 0, Mass[Fu,3]}}'}}
******************************
******************************
{'mu2': {'Description': 'SM Mu Parameter', 'OutputName': 'm2SM'}}
******************************
******************************
{'Yd': {'Description': 'Down-Yukawa-Coupling', 'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}}
******************************
*************

In [30]:
ParameterDefinitions

{'Properties': {'AlphaS': {'Description': 'Alpha Strong'},
  'Gf': {'Description': "Fermi's constant"},
  'ThetaW': {'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]',
   'Description': 'Weinberg-Angle'},
  'Ud': {'Description': 'Right-Down-Mixing-Matrix'},
  'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'},
  'Uu': {'Description': 'Right-Up-Mixing-Matrix'},
  'Vd': {'Description': 'Left-Down-Mixing-Matrix'},
  'Ve': {'Description': 'Left-Lepton-Mixing-Matrix'},
  'Vu': {'Description': 'Left-Up-Mixing-Matrix'},
  'Yd': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}',
   'Description': 'Down-Yukawa-Coupling'},
  'Ye': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}',
   'Description': 'Lepton-Yukawa-Coupling'},
  'Yu': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fu,1],0,0 },\n                {0, Mass[Fu,2],0},\n                {0, 0

In [31]:
pd.DataFrame(ParameterDefinitions)

,Properties
AlphaS,{'Description': 'Alpha Strong'}
Gf,{'Description': 'Fermi's constant'}
ThetaW,"{'Description': 'Weinberg-Angle', 'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]'}"
Ud,{'Description': 'Right-Down-Mixing-Matrix'}
Ue,{'Description': 'Right-Lepton-Mixing-Matrix'}
Uu,{'Description': 'Right-Up-Mixing-Matrix'}
Vd,{'Description': 'Left-Down-Mixing-Matrix'}
Ve,{'Description': 'Left-Lepton-Mixing-Matrix'}
Vu,{'Description': 'Left-Up-Mixing-Matrix'}
Yd,"{'Description': 'Down-Yukawa-Coupling', 'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 }, {0, Mass[Fd,2],0}, {0, 0, Mass[Fd,3]}}'}"
